In [ ]:
import os
import shutil
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [ ]:

# 특정 색상 계열로 부드럽게 제한하는 함수
def apply_soft_color_tone(image, target_colors, blend_factor_range=(0.2, 0.4)):
    image = image.astype(np.float32) / 255.0  # 0~1로 정규화
    h, w = image.shape[:2]

    # 랜덤하게 하나의 색상 계열을 선택
    chosen_color = (
        np.array(target_colors[np.random.randint(len(target_colors))], dtype=np.float32)
        / 255.0
    )
    chosen_color = np.ones((h, w, 3), dtype=np.float32) * chosen_color

    # 부드러운 색상 혼합
    blend_factor = np.random.uniform(*blend_factor_range)
    image = cv2.addWeighted(image, 1 - blend_factor, chosen_color, blend_factor, 0)

    return (image * 255).astype(np.uint8)


# 데이터 증강 함수
def augment_and_save(image, mask, target_dir, base_filename, start_idx, num_augmentations):
    target_colors = [
        (139, 69, 19),  # 갈색
        (255, 223, 0),  # 노란색
        (220, 20, 60),  # 붉은색
        (0, 128, 0),  # 녹색
        (0, 0, 0),  # 검은색
    ]

    transform = A.Compose(
        [
            A.OneOf(
                [
                    A.RandomRotate90(),  # 90도 회전
                    A.Flip(),  # 좌우 반전
                    A.Transpose(),  # 대각선 반전
                ],
                p=0.5,
            ),
            A.OneOf(
                [
                    A.GaussianBlur(blur_limit=3, p=0.5),  # 가우시안 블러
                    A.MotionBlur(blur_limit=3, p=0.5),  # 모션 블러
                    A.MedianBlur(blur_limit=3, p=0.5),  # 미디안 블러
                ],
                p=0.5,
            ),
            A.ShiftScaleRotate(
                shift_limit=0.1,
                scale_limit=0.1,
                rotate_limit=20,
                border_mode=cv2.BORDER_CONSTANT,
                p=0.5,
            ),  # 이동, 확대, 회전
            ToTensorV2(),  # PyTorch 텐서로 변환
        ]
    )
    print(f"num_augmentations : {num_augmentations}")
    
    for i in range(num_augmentations):
        augmented = transform(image=image)["image"]
        augmented_image = augmented.numpy().transpose(1, 2, 0)  # 이미지 차원 변환 (C, H, W) -> (H, W, C)

        # 디버깅: 변환 후 이미지 확인
        plt.imshow(augmented_image)
        plt.title("Augmented Image")
        plt.show()

        # 부드러운 색상 혼합 적용 (이미지에만 적용)
        augmented_image = apply_soft_color_tone(augmented_image, target_colors)

        # 디버깅: 색상 혼합 후 이미지 확인
        plt.imshow(augmented_image)
        plt.title("Color-Toned Image")
        plt.show()

        new_filename_img = f"{base_filename}_aug_{start_idx + i}.jpg"
        new_filename_mask = f"{base_filename}_mask_aug_{start_idx + i}.png"
        new_filepath_img = os.path.join(target_dir, new_filename_img)
        new_filepath_mask = os.path.join(target_dir, new_filename_mask)

        # RGB에서 BGR로 변환하여 저장
        cv2.imwrite(new_filepath_img, augmented_image[:, :, ::-1])  # BGR로 변환하여 저장

        # 마스크는 그대로 저장
        cv2.imwrite(new_filepath_mask, mask)


# 데이터 복사 함수 (300개 이상인 경우)
def copy_images(class_dir, output_class_dir):
    images = [
        f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))
    ]
    for image_file in images:
        src_path = os.path.join(class_dir, image_file)
        dst_path = os.path.join(output_class_dir, image_file)
        shutil.copy2(src_path, dst_path)  # 메타데이터까지 복사


# 데이터 증강을 적용할 함수
def apply_augmentation(dataset_dir, output_dir, target_class, target_count):
    class_dir = os.path.join(dataset_dir, target_class)
    output_class_dir = os.path.join(output_dir, target_class)
    os.makedirs(output_class_dir, exist_ok=True)
    images = [
        f for f in os.listdir(class_dir) if f.endswith('.jpg') and os.path.isfile(os.path.join(class_dir, f))
    ]
    num_images = len(images)

    if num_images >= target_count:
        print(
            f"{target_class} 클래스는 이미 {target_count}장 이상 존재합니다. 이미지를 복사합니다."
        )
        copy_images(class_dir, output_class_dir)
        return

    num_augmentations = target_count - num_images

    print(
        f"{target_class} 클래스에 대해 {num_augmentations}개의 증강 이미지 생성 중..."
    )

    for i, image_file in enumerate(tqdm(images)):
        image_path = os.path.join(class_dir, image_file)
        mask_path = os.path.join(class_dir, os.path.splitext(image_file)[0] + '_mask.png')

        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # 마스크는 그레이스케일로 읽기
        augment_and_save(
            image,
            mask,
            output_class_dir,
            os.path.splitext(image_file)[0],
            i * max(num_augmentations // num_images, 1),
            max(num_augmentations // num_images, 1),
        )


In [ ]:

# 데이터셋의 각 클래스에 대해 증강 적용
dataset_dir = 'dataset_noaug/train'  # 입력 데이터셋 경로 설정
output_dir = 'dataset_aug_0816/train'  # 증강된 데이터를 저장할 경로 설정
target_classes = ["1", "2", "3", "4", "5", "6", "7"]
target_count = 300  # 각 클래스당 목표 이미지 수

for target_class in target_classes:
    apply_augmentation(dataset_dir, output_dir, target_class, target_count)